Criando um sistema de recomendação de filmes usando Machine Learning, processo semelhante as plataformas da Netflix, Apple Music, Spotify, entre outros.

fonte: https://www.kaggle.com/code/alyssonbispopereira/recomenda-o-de-filmes-ptbr/data

crédito: https://www.youtube.com/watch?v=sDI3ntGFbSI&t=2029s&ab_channel=NerddosDados

In [12]:
import pandas as pd
import numpy as np

Importando os arquivos com os filmes e avaliações.

In [13]:
filmes = pd.read_csv('movies_metadata.csv', low_memory=False)
filmes.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [14]:
avaliacoes = pd.read_csv('ratings.csv')
avaliacoes.head(5)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


Fazendo o pré processamento dos dados do DataFrame "filmes". Vamos filtrar somente as colunas necessárias e renomear o nome das vairáveis.

In [15]:
filmes = filmes [['id', 'original_title', 'original_language', 'vote_count']]
filmes.rename(columns = {'id': 'ID_FILME', 'original_title': 'TITULO',
                         'original_language': 'LINGUAGEM',
                         'vote_count': 'QT_AVALIACOES'}, inplace=True)
filmes.head()

<ipython-input-15-8eea0f4b05b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes.rename(columns = {'id': 'ID_FILME', 'original_title': 'TITULO',


,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
2,15602,Grumpier Old Men,en,92.0
3,31357,Waiting to Exhale,en,34.0
4,11862,Father of the Bride Part II,en,173.0


Fazendo o pré processamento dos dados do DataFrame "avaliacoes". Vamos filtrar somente as colunas necessárias e renomear o nome das vairáveis.

In [17]:
avaliacoes = avaliacoes [['userId', 'movieId', 'rating']]
avaliacoes.rename(columns = {'userId': 'ID_USUARIO', 'movieId': 'ID_FILME',
                             'rating': 'AVALIACAO'}, inplace=True)
avaliacoes.head()

,ID_USUARIO,ID_FILME,AVALIACAO
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


verificando e removendo os valores nulos

In [18]:
filmes.isna().sum()

ID_FILME          0
TITULO            0
LINGUAGEM        11
QT_AVALIACOES     6
dtype: int64

In [19]:
filmes.dropna(inplace=True)

In [20]:
filmes.isna().sum()

ID_FILME         0
TITULO           0
LINGUAGEM        0
QT_AVALIACOES    0
dtype: int64

In [21]:
avaliacoes.isna().sum()

ID_USUARIO    0
ID_FILME      0
AVALIACAO     0
dtype: int64

Verificando a quantidade de avaliações por usuário

In [22]:
avaliacoes['ID_USUARIO'].value_counts()

45811     18276
8659       9279
179792     7515
228291     7410
98415      6094
          ...  
50849         1
91351         1
141715        1
181494        1
163066        1
Name: ID_USUARIO, Length: 238538, dtype: int64